# ETL with EEOC
# Khrizel Solano and Kelli Okuji Wilson

# Extract 

In [2]:
#pip install mysqlclient
import fnmatch
# from regexdict import regexdict
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [59]:
# Store CSV into DataFrame

In [3]:
csv_file = "./Resources/year16_nac2.csv"
eeoc_nac2_df = pd.read_csv(csv_file, sep=';').dropna(how='all')
eeoc_nac2_df.sample(5)
#eeoc_db.year16_state_nac2.csv

,NAC2_code,TOTAL_UNITS,TOTAL10,WHM1,WHM2,WHM3,WHM4,WHM5,WHM6,WHM7,...,TOMRF7,TOMRF8,TOMRF9,TOMRF1_2,NAC2_Label,i,SUMCOUNT,MISSCOUNT,SMALLEST,INDEX
12,51,8407,1731043,21470,271515,64311,63472,64590,52241,13782,...,213,110.0,1456,1572,Information,141,0,0,NaN,NaN
22,81,5700,879176,11322,38777,17266,17515,15542,26124,21644,...,317,959.0,5847,708,Other Services (except Public Administration),141,0,0,NaN,NaN
9,45,19329,3669678,6785,56084,13689,312296,153464,34353,47627,...,1055,4970.0,2706,1922,Retail Trade,141,0,0,NaN,NaN
2,22,2564,466109,5883,64738,25510,1645,9568,87506,21627,...,20,9.0,4,148,Utilities,141,6,4,NaN,NaN
1,21,1935,346103,5039,37459,14647,2088,3999,58589,43730,...,32,11.0,8,94,"Mining, Quarrying, and Oil and Gas Extraction",141,9,6,NaN,NaN


In [4]:
eeoc_kaggle_csv_file = "./Resources/Reveal_EEO1_for_2016.csv"
eeoc_kaggle_df = pd.read_csv(eeoc_kaggle_csv_file).dropna(how='all')
eeoc_kaggle_df.sample(5)
#Reveal_EEO1_for_2016.csv

,company,year,race,gender,job_category,count
699,Apple,2016,Two_or_more_races,female,Technicians,215
518,Airbnb,2016,Two_or_more_races,female,Professionals,10
3996,PayPal,2016,Black_or_African_American,male,Executives,1
1487,HP Inc.,2016,Black_or_African_American,male,Previous_totals,2551
438,Airbnb,2016,American_Indian_Alaskan_Native,male,Craft workers,0


## Transform

* Transform the EEOC table from wide to narrow so it's easier to join with the Kaggle data set

* Create data dictionaries for race, gender, and job category in EEOC data set to match the Kaggle data set values

* Clean up the values in the job category in the Kaggle data set so it matches with the EEOC data set values

* Drop nulls and total/subtotal lines in Kaggle data set

* Create new tables by joining the EEOC and Kaggle data sets


## Write-up
Our final table does not include a join/merge of the two transformed datasets from Kaggle and the EEOC. After transforming the datasets, we encountered an obstacle in joining the datasets on a common identifer. The reason that we encountered this issue is due to the fact that the Kaggle data set was already aggregated by company. The EEOC data was parsed out to create sub-categories of the aggregated data set by gender, job category, and race. However, we could not join on aggregated data.

If we were to correct the issue going forward, we would identify a dataset that reported individual instances such as by employee and his/her individual characteristics (gender, job category, and race) so we could more easily manipulate the data through aggreggation and join it to more broad data set such as comparing to state or national measures.

We currently have appended to the manipulated Kaggle dataset the California, Tech sector aggregated data in order to make a tentaive comparison of how Silicon Valley tech employment compares to the California state tech sector measures.

In [76]:
# Create new data sets with select columns

In [5]:
eeoc_nac2_All_df = eeoc_nac2_df[['NAC2_code','NAC2_Label','WHM1','WHM2','WHM3','WHM4','WHM5','WHM6','WHM7','WHM8','WHM9','WHF1','WHF2','WHF3','WHF4','WHF5','WHF6','WHF7','WHF8','WHF9','BLKM1','BLKM2','BLKM3','BLKM4','BLKM5','BLKM6','BLKM7','BLKM8','BLKM9','BLKF1','BLKF2','BLKF3','BLKF4','BLKF5','BLKF6','BLKF7','BLKF8','BLKF9','HISPM1','HISPM2','HISPM3','HISPM4','HISPM5','HISPM6','HISPM7','HISPM8','HISPM9','HISPF1','HISPF2','HISPF3','HISPF4','HISPF5','HISPF6','HISPF7','HISPF8','HISPF9','ASIANM1','ASIANM2','ASIANM3','ASIANM4','ASIANM5','ASIANM6','ASIANM7','ASIANM8','ASIANM9','ASIANF1','ASIANF2','ASIANF3','ASIANF4','ASIANF5','ASIANF6','ASIANF7','ASIANF8','ASIANF9','AIANM1','AIANM2','AIANM3','AIANM4','AIANM5','AIANM6','AIANM7','AIANM8','AIANM9','AIANF1','AIANF2','AIANF3','AIANF4','AIANF5','AIANF6','AIANF7','AIANF8','AIANF9','NHOPIM1','NHOPIM2','NHOPIM3','NHOPIM4','NHOPIM5','NHOPIM6','NHOPIM7','NHOPIM8','NHOPIM9','NHOPIF1','NHOPIF2','NHOPIF3','NHOPIF4','NHOPIF5','NHOPIF6','NHOPIF7','NHOPIF8','NHOPIF9','TOMRM1','TOMRM2','TOMRM3','TOMRM4','TOMRM5','TOMRM6','TOMRM7','TOMRM8','TOMRM9','TOMRF1','TOMRF2','TOMRF3','TOMRF4','TOMRF5','TOMRF6','TOMRF7','TOMRF8','TOMRF9']]
eeoc_nac2_All_df.head()

,NAC2_code,NAC2_Label,WHM1,WHM2,WHM3,WHM4,WHM5,WHM6,WHM7,WHM8,...,TOMRM9,TOMRF1,TOMRF2,TOMRF3,TOMRF4,TOMRF5,TOMRF6,TOMRF7,TOMRF8,TOMRF9
0,11,"Agriculture, Forestry, Fishing and Hunting",3311,7254,2369,4740,2561,5636,12178,15711,...,71,6,58,26,35.0,100,5,68,871.0,35
1,21,"Mining, Quarrying, and Oil and Gas Extraction",5039,37459,14647,2088,3999,58589,43730,15476,...,12,6,322,71,10.0,214,13,32,11.0,8
2,22,Utilities,5883,64738,25510,1645,9568,87506,21627,6961,...,45,13,600,63,22.0,682,28,20,9.0,4
3,23,Construction,26632,83888,37250,21458,24350,393797,91062,114587,...,173,43,529,101,251.0,1436,274,88,196.0,52
4,31,Manufacturing,15623,39393,18702,39607,17628,83772,174938,80811,...,731,37,621,213,462.0,816,128,1300,1630.0,755


In [ ]:
# Convert wide to narrow table using melt

In [6]:
eeoc_nac2_All_unpivot_df=pd.melt(eeoc_nac2_All_df, id_vars=['NAC2_code','NAC2_Label'], var_name="EEOC_Code", value_name="count").dropna(how='all')
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count
1565,32,Manufacturing,ASIANF3,1929.0
399,54,"Professional, Scientific, and Technical Services",WHF8,7928.0
1576,55,Management of Companies and Enterprises,ASIANF3,783.0
2030,53,Real Estate and Rental and Leasing,AIANF4,83.0
55,42,Wholesale Trade,WHM3,25811.0


In [ ]:
#Transform EEOC data

In [7]:
eeoc_nac2_All_unpivot_df['gender'] = eeoc_nac2_All_unpivot_df['EEOC_Code'].str[-2].replace({'M' : 'Male', 'F' : 'Female'})
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender
2161,21,"Mining, Quarrying, and Oil and Gas Extraction",NHOPIM1,NaN,Male
2328,11,"Agriculture, Forestry, Fishing and Hunting",NHOPIM8,444.0,Male
1301,32,Manufacturing,ASIANM1,1866.0,Male
1174,81,Other Services (except Public Administration),HISPF4,3355.0,Female
150,33,Manufacturing,WHM7,716817.0,Male


In [19]:
eeoc_nac2_All_unpivot_df['job_category'] = eeoc_nac2_All_unpivot_df['EEOC_Code'].str[-1].replace({'1' : 'Senior OFF AND MGRS', '2' : 'PROF', '3' : 'TECH', '4' : 'SALE', '5' : 'CLERICALS', '6' : 'CRAFT', '7' : 'OPERS','8' : 'LABORS', '9' : 'Service'}).dropna(how='all')
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category_coded,race,job_category
174,33,Manufacturing,WHM8,166012.0,Male,LABORS,White,LABORS
3018,61,Educational Services,TOMRF9,855.0,Female,Service,Two_or_more_races,Service
31,42,Wholesale Trade,WHM2,75561.0,Male,PROF,White,PROF
428,71,"Arts, Entertainment, and Recreation",WHF9,149761.0,Female,Service,White,Service
2365,52,Finance and Insurance,NHOPIM9,24.0,Male,Service,Native_Hawaiian_or_Pacific_Islander,Service


In [21]:
eeoc_nac2_All_unpivot_df['race'] = eeoc_nac2_All_unpivot_df['EEOC_Code'].str[:-2].replace({'WH' : 'White', 'BLK' : 'Black', 'HISP': 'Hispanic_or_Latino','ASIAN' : 'Asian','AIAN' : 'American_Indian_Alaskan_Native','NHOPI' : 'Native_Hawaiian_or_Pacific_Islander','TOMR' : 'Two_or_more_races'}).dropna(how='all')
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category_coded,race,job_category
2626,48,Transportation and Warehousing,TOMRM2,1461.0,Male,PROF,Two_or_more_races,PROF
3009,45,Retail Trade,TOMRF9,2706.0,Female,Service,Two_or_more_races,Service
523,62,Health Care and Social Assistance,BLKM4,1311.0,Male,SALE,Black,SALE
1341,72,Accommodation and Food Services,ASIANM2,2288.0,Male,PROF,Asian,PROF
3013,52,Finance and Insurance,TOMRF9,103.0,Female,Service,Two_or_more_races,Service


In [ ]:
#Transform Kaggle data

In [22]:
eeoc_kaggle_df.sample(5)

,company,year,race,gender,job_category,count
34,23andMe,2016,White,Male,Totals,102
3705,Uber,2016,White,Female,Service,0
4212,NetApp,2016,Hispanic_or_Latino,Female,Senior OFF AND MGRS,1
2113,Intuit,2016,Asian,Female,Senior OFF AND MGRS,83
3172,Salesforce,2016,Black_or_African_American,Female,SALE,19


In [23]:
eeoc_kaggle_df['gender'] = eeoc_kaggle_df['gender'].replace({'male' : 'Male', 'female' : 'Female'}).dropna(how='all')
eeoc_kaggle_df.sample(5)

,company,year,race,gender,job_category,count
3991,PayPal,2016,White,Male,OPERS,0
172,23andMe,2016,Overall_totals,NaN,SALE,0
4006,PayPal,2016,Black_or_African_American,Male,Totals,325
3076,Salesforce,2016,Hispanic_or_Latino,Female,SALE,35
2075,Intuit,2016,Two_or_more_races,Male,Previous_totals,77


In [24]:
eeoc_kaggle_df['job_category'] = eeoc_kaggle_df['job_category'].replace({'Executives' : 'Senior OFF AND MGRS', 'Managers' : 'Senior OFF AND MGRS', 'Professionals' : 'PROF', 'Technicians' : 'TECH', 'Sales workers' : 'SALE', 'Administrative support' : 'CLERICALS', 'Craft workers' : 'CRAFT', 'operatives' : 'OPERS','laborers and helpers' : 'LABORS', 'Service workers' : 'Service'}).dropna(how=all)
eeoc_kaggle_df.sample(5)

,company,year,race,gender,job_category,count
3014,Pinterest,2016,Asian,Female,PROF,123
2275,LinkedIn,2016,Black_or_African_American,Female,OPERS,0
3734,Uber,2016,Asian,Female,PROF,430
1590,HP Inc.,2016,American_Indian_Alaskan_Native,Female,CRAFT,0
2056,Intuit,2016,American_Indian_Alaskan_Native,Male,SALE,3


In [25]:
eeoc_kaggle_df_clean = eeoc_kaggle_df[(eeoc_kaggle_df["job_category"]!= "Previous_totals") & (eeoc_kaggle_df["job_category"]!= "Overall_totals")&(eeoc_kaggle_df["race"]!= "Previous_totals") & (eeoc_kaggle_df["race"]!= "Overall_totals")]
eeoc_kaggle_df_clean.sample(5)

,company,year,race,gender,job_category,count
3436,Twitter,2016,Hispanic_or_Latino,Female,SALE,10
632,Apple,2016,Two_or_more_races,Male,LABORS,0
2258,LinkedIn,2016,White,Female,PROF,716
797,Cisco,2016,American_Indian_Alaskan_Native,Male,CLERICALS,0
4082,PayPal,2016,Native_Hawaiian_or_Pacific_Islander,Female,PROF,5


In [ ]:
#join these 2 data sets: EEOC and Kaggle

In [26]:
eeoc_kaggle_df_clean.sample(5)

,company,year,race,gender,job_category,count
3378,Square,2016,Asian,Female,CRAFT,0
1001,eBay,2016,White,Female,CLERICALS,560
2950,Pinterest,2016,Asian,Male,Totals,236
3352,Square,2016,Black_or_African_American,Female,SALE,0
3065,Salesforce,2016,Hispanic_or_Latino,Male,CLERICALS,11


In [27]:
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category_coded,race,job_category
2182,81,Other Services (except Public Administration),NHOPIM1,19.0,Male,Senior OFF AND MGRS,Native_Hawaiian_or_Pacific_Islander,Senior OFF AND MGRS
2011,62,Health Care and Social Assistance,AIANF3,6590.0,Female,TECH,American_Indian_Alaskan_Native,TECH
1070,53,Real Estate and Rental and Leasing,HISPM9,7064.0,Male,Service,Hispanic_or_Latino,Service
2973,72,Accommodation and Food Services,TOMRF7,343.0,Female,OPERS,Two_or_more_races,OPERS
188,71,"Arts, Entertainment, and Recreation",WHM8,27554.0,Male,LABORS,White,LABORS


In [47]:
eeoc_tech=eeoc_nac2_All_unpivot_df[(eeoc_nac2_All_unpivot_df['NAC2_Label']  =='Professional, Scientific, and Technical Services')]
eeoc_tech.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category_coded,race,job_category
2079,54,"Professional, Scientific, and Technical Services",AIANF6,71.0,Female,CRAFT,American_Indian_Alaskan_Native,CRAFT
1071,54,"Professional, Scientific, and Technical Services",HISPM9,4604.0,Male,Service,Hispanic_or_Latino,Service
1335,54,"Professional, Scientific, and Technical Services",ASIANM2,210965.0,Male,PROF,Asian,PROF
2103,54,"Professional, Scientific, and Technical Services",AIANF7,205.0,Female,OPERS,American_Indian_Alaskan_Native,OPERS
567,54,"Professional, Scientific, and Technical Services",BLKM6,5240.0,Male,CRAFT,Black,CRAFT


In [48]:
eeoc_female=eeoc_nac2_All_unpivot_df[(eeoc_nac2_All_unpivot_df['NAC2_Label']  =='Professional, Scientific, and Technical Services') & (eeoc_nac2_All_unpivot_df['gender']  =='Female' )]
eeoc_female.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category_coded,race,job_category
1167,54,"Professional, Scientific, and Technical Services",HISPF4,4617.0,Female,SALE,Hispanic_or_Latino,SALE
735,54,"Professional, Scientific, and Technical Services",BLKF4,5518.0,Female,SALE,Black,SALE
303,54,"Professional, Scientific, and Technical Services",WHF4,44438.0,Female,SALE,White,SALE
2055,54,"Professional, Scientific, and Technical Services",AIANF5,1883.0,Female,CLERICALS,American_Indian_Alaskan_Native,CLERICALS
2919,54,"Professional, Scientific, and Technical Services",TOMRF5,10124.0,Female,CLERICALS,Two_or_more_races,CLERICALS


In [61]:
kaggle_female=eeoc_kaggle_df_clean.groupby([eeoc_kaggle_df_clean['gender']])['gender'].count()
kaggle_female

gender
Female    1925
Male      1925
Name: gender, dtype: int64

In [36]:
# gender_merge = pd.merge(eeoc_kaggle_df_clean, eeoc_nac2_All_unpivot_df, on='gender', how = 'inner')
# gender_merge.sample(10)

In [38]:
# job_category_merge = pd.merge(eeoc_kaggle_df_clean, eeoc_nac2_All_unpivot_df, on='job_category', how='inner')
# job_category_merge.sample(10)

In [ ]:
df1.merge(df2, left_on='lkey', right_on='rkey')

In [ ]:
#connect to local database

In [90]:
rds_connection_string = "root:mysqlp5ssw0rd*@127.0.0.1/eeoc_db" #hide this
engine = create_engine(f'mysql://{rds_connection_string}')

In [ ]:
#create tables

In [ ]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

In [ ]:
#connect to local database

In [25]:
rds_connection_string = "root:jtf@nfF1749@127.0.0.1/customer_db" #hide this
engine = create_engine(f'mysql://{rds_connection_string}')

In [26]:
# Check for tables

In [27]:
engine.table_names()

['customer_location', 'customer_name']

In [ ]:
# Use pandas to load csv converted DataFrame into database

In [28]:
eeoc_kaggle_df_clean_df.to_sql(name='eeoc_kaggle_df_clean', con=engine, if_exists='append', index=False)

In [29]:
# Use pandas to load json converted DataFrame into database

In [30]:
eeoc_nac2_All_unpivot_df_df.to_sql(name='eeoc_nac2_All_unpivot_df', con=engine, if_exists='append', index=False)

In [31]:
# Confirm data has been added by querying the customer_name table
# NOTE: can also check using pgAdmin

In [32]:
pd.read_sql_query('select * eeoc_kaggle_df_clean', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


In [33]:
# Confirm data has been added by querying the customer_location table

In [34]:
pd.read_sql_query('select * from eeoc_nac2_All_unpivot_df', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York
